# Feature Reduction using Bio Inspired Algorithms

In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

from sklearn.exceptions import UndefinedMetricWarning
import warnings
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import random



warnings.filterwarnings("ignore")
warnings.resetwarnings()
warnings.filterwarnings("ignore", category=FutureWarning)


In [4]:
df=pd.read_csv("/Users/saikarthik/Desktop/UROP/data.csv")

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [5]:
df.dropna(inplace=True)


In [6]:
df.drop_duplicates(inplace=True)


In [33]:
c0=0
c1=0
for i in  df["Label"]:
    if i== 0 :
        c0=c0+1
    elif i==1:
        c1=c1+1
print(c0/c1)            
        

6.975562072336266


In [7]:
for column in df.columns:
    if df[column].dtype == 'object':  
        df[column] = df[column].replace(to_replace='.*', value=0, regex=True)

In [8]:
cols_to_drop = [col for col in df.columns if df[col].nunique() == 1]
df = df.drop(columns=cols_to_drop)


In [33]:
df.columns

Index(['SrcBytes', 'DstBytes', 'SrcLoad', 'DstLoad', 'SIntPkt', 'DIntPkt',
       'SIntPktAct', 'SrcJitter', 'DstJitter', 'sMaxPktSz', 'dMaxPktSz',
       'sMinPktSz', 'Dur', 'TotPkts', 'TotBytes', 'Load', 'Loss', 'pLoss',
       'pSrcLoss', 'pDstLoss', 'Rate', 'Packet_num', 'Temp', 'SpO2',
       'Pulse_Rate', 'SYS', 'DIA', 'Heart_rate', 'Resp_Rate', 'ST', 'Label'],
      dtype='object')

# Random Forest 

### Before GA and HC

In [13]:
X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)

start_time = time.time()

clf.fit(X_train, y_train)

end_time = time.time()

y_pred = clf.predict(X_test)

execution_time = end_time - start_time

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Execution Time: {execution_time:.2f} seconds")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Execution Time: 2.00 seconds
Accuracy: 0.9347426470588235
Precision: 0.981042654028436
Recall: 0.49759615384615385
F1-Score: 0.6602870813397129
Confusion Matrix:
 [[2844    4]
 [ 209  207]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96      2848
           1       0.98      0.50      0.66       416

    accuracy                           0.93      3264
   macro avg       0.96      0.75      0.81      3264
weighted avg       0.94      0.93      0.93      3264



### After applying Genetic Algorithm

In [ ]:


X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

population_size = 50
generations = 100
mutation_rate = 0.01

def evaluate_features(features):
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train.iloc[:, features], y_train)
    y_pred = clf.predict(X_test.iloc[:, features])
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

population = [np.random.choice([0, 1], size=X_train.shape[1]) for _ in range(population_size)]



for generation in range(generations):
    fitness_scores = [evaluate_features(individual) for individual in population]
    num_parents = int(population_size * 0.2)
    parents = np.argsort(fitness_scores)[-num_parents]
    parents = parents.tolist()  
    new_population = []
    for _ in range(population_size - num_parents):
        parent1 = population[np.random.choice(parents)]
        parent2 = population[np.random.choice(parents)]
        crossover_point = np.random.randint(0, X_train.shape[1])
        child = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
        mutation = np.random.rand(X_train.shape[1]) < mutation_rate
        child = (child + mutation) % 2
        new_population.append(child)
    population = np.vstack((population[parents], new_population))

best_individual = population[np.argmax(fitness_scores)]
selected_features = np.where(best_individual == 1)[0]

start_time_with_ga = time.time()
clf_with_ga = RandomForestClassifier(n_estimators=100, random_state=42)
clf_with_ga.fit(X_train.iloc[:, selected_features], y_train)
y_pred_with_ga = clf_with_ga.predict(X_test.iloc[:, selected_features])
end_time_with_ga = time.time()

time_taken_with_ga = end_time_with_ga - start_time_with_ga

confusion_matrix_with_ga = confusion_matrix(y_test, y_pred_with_ga)

print("Time taken for Model with Genetic Algorithm Feature Selection:", time_taken_with_ga, "seconds")


print("\nConfusion Matrix for Model with Genetic Algorithm Feature Selection:")
print(confusion_matrix_with_ga)




print("\nConfusion Matrix for Model with Genetic Algorithm Feature Selection:")
print(confusion_matrix_with_ga)



accuracy_with_ga = accuracy_score(y_test, y_pred_with_ga)
precision_with_ga = precision_score(y_test, y_pred_with_ga)
recall_with_ga = recall_score(y_test, y_pred_with_ga)
f1_score_with_ga = f1_score(y_test, y_pred_with_ga)


print("\nModel with Genetic Algorithm Feature Selection Metrics:")
print("Accuracy (with GA):", accuracy_with_ga)
print("Precision (with GA):", precision_with_ga)
print("Recall (with GA):", recall_with_ga)
print("F1 Score (with GA):", f1_score_with_ga)


### After applying Hill climb on Random forest

In [23]:
X = df.drop('Label', axis=1)
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.to_numpy()  
X_test = X_test.to_numpy()  

def objective_function(features):
    start_time = time.time()
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train[:, features], y_train)
    y_pred = clf.predict(X_test[:, features])
    end_time = time.time()

    return accuracy_score(y_test, y_pred)

def hill_climbing_feature_selection(num_features, num_iterations):
    current_features = list(range(num_features))
    current_score = objective_function(current_features)

    for _ in range(num_iterations):
        neighbor = current_features.copy()
        if len(neighbor) > 1:  
            feature_to_remove = random.choice(neighbor)
            neighbor.remove(feature_to_remove)
        else:
            features_not_in_subset = list(set(range(num_features)) - set(neighbor))
            if features_not_in_subset:
                feature_to_add = random.choice(features_not_in_subset)
                neighbor.append(feature_to_add)
        neighbor_score = objective_function(neighbor)
        if neighbor_score > current_score:
            current_features = neighbor
            current_score = neighbor_score

    return current_features, current_score

num_features = X_train.shape[1]
num_iterations = 100

selected_features, best_score = hill_climbing_feature_selection(num_features, num_iterations)
elapsed_time = end_time - start_time

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train[:, selected_features], y_train)
y_pred = clf.predict(X_test[:, selected_features])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

print("Selected Features:", selected_features)
print("Best Score:", best_score)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", confusion)
print("Time taken:", elapsed_time, "seconds")

Selected Features: [0, 1, 2, 3, 10, 11, 12, 13, 14, 15, 18, 20, 21, 24, 25, 26, 27, 28, 29, 31, 32]
Best Score: 0.9589460784313726
Accuracy: 0.9589460784313726
Precision: 0.9595113439439142
Recall: 0.9589460784313726
F1 Score: 0.9559702794086331
Confusion Matrix:
 [[2840    8]
 [ 126  290]]
Time taken: 2.0015292167663574 seconds


# --------------------------------------------------
# SVM

### Before GA and HC

In [24]:
from sklearn.svm import SVC

svm = SVC(kernel='linear', random_state=42)

start_time = time.time()

svm.fit(X_train, y_train)

end_time = time.time()

y_pred_svm = svm.predict(X_test)

execution_time = end_time - start_time

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_score_svm = f1_score(y_test, y_pred_svm)
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
classification_rep_svm = classification_report(y_test, y_pred_svm)

print(f"Execution Time (SVM): {execution_time:.2f} seconds")
print("Accuracy (SVM):", accuracy_svm)
print("Precision (SVM):", precision_svm)
print("Recall (SVM):", recall_svm)
print("F1-Score (SVM):", f1_score_svm)
print("Confusion Matrix (SVM):\n", conf_matrix_svm)
print("Classification Report (SVM):\n", classification_rep_svm)


Execution Time (SVM): 364.24 seconds
Accuracy (SVM): 0.9283088235294118
Precision (SVM): 0.989247311827957
Recall (SVM): 0.4423076923076923
F1-Score (SVM): 0.6112956810631229
Confusion Matrix (SVM):
 [[2846    2]
 [ 232  184]]
Classification Report (SVM):
               precision    recall  f1-score   support

           0       0.92      1.00      0.96      2848
           1       0.99      0.44      0.61       416

    accuracy                           0.93      3264
   macro avg       0.96      0.72      0.79      3264
weighted avg       0.93      0.93      0.92      3264



In [27]:
X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

population_size = 50
generations = 100
mutation_rate = 0.01

def evaluate_features(features):
    clf = SVC(kernel='linear', random_state=42)
    clf.fit(X_train.iloc[:, features], y_train)
    y_pred = clf.predict(X_test.iloc[:, features])
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

population = [np.random.choice([0, 1], size=X_train.shape[1]) for _ in range(population_size)]

for generation in range(generations):
    fitness_scores = [evaluate_features(individual) for individual in population]
    num_parents = int(population_size * 0.2)
    parents = np.argsort(fitness_scores)[-num_parents]
    parents = parents.tolist()
    new_population = []
    for _ in range(population_size - num_parents):
        parent1 = population[np.random.choice(parents)]
        parent2 = population[np.random.choice(parents)]
        crossover_point = np.random.randint(0, X_train.shape[1])
        child = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
        mutation = np.random.rand(X_train.shape[1]) < mutation_rate
        child = (child + mutation) % 2
        new_population.append(child)
    population = np.vstack((population[parents], new_population))

best_individual = population[np.argmax(fitness_scores)]
selected_features = np.where(best_individual == 1)[0]

start_time_with_ga = time.time()
clf_with_ga = SVC(kernel='linear', random_state=42)
clf_with_ga.fit(X_train.iloc[:, selected_features], y_train)
y_pred_with_ga = clf_with_ga.predict(X_test.iloc[:, selected_features])
end_time_with_ga = time.time()

time_taken_with_ga = end_time_with_ga - start_time_with_ga

confusion_matrix_with_ga = confusion_matrix(y_test, y_pred_with_ga)

print("Time taken for Model with Genetic Algorithm Feature Selection:", time_taken_with_ga, "seconds")

print("\nConfusion Matrix for Model with Genetic Algorithm Feature Selection:")
print(confusion_matrix_with_ga)

accuracy_with_ga = accuracy_score(y_test, y_pred_with_ga)
precision_with_ga = precision_score(y_test, y_pred_with_ga)
recall_with_ga = recall_score(y_test, y_pred_with_ga)
f1_score_with_ga = f1_score(y_test, y_pred_with_ga)

print("\nModel with Genetic Algorithm Feature Selection Metrics:")
print("Accuracy (with GA):", accuracy_with_ga)
print("Precision (with GA):", precision_with_ga)
print("Recall (with GA):", recall_with_ga)
print("F1 Score (with GA):", f1_score_with_ga)

Time taken for Model with Genetic Algorithm Feature Selection: 478.169570684433 seconds

Confusion Matrix for Model with Genetic Algorithm Feature Selection:
[[2840    8]
 [ 234  182]]

Model with Genetic Algorithm Feature Selection Metrics:
Accuracy (with GA): 0.9258578431372549
Precision (with GA): 0.9578947368421052
Recall (with GA): 0.4375
F1 Score (with GA): 0.6006600660066007


### After Hill Climb 

In [28]:
X = df.drop('Label', axis=1)
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

start_time = time.time()

def objective_function(features):
    clf = SVC(kernel='linear', random_state=42)  
    clf.fit(X_train[:, features], y_train)
    y_pred = clf.predict(X_test[:, features])

    return accuracy_score(y_test, y_pred)

def hill_climbing_feature_selection(num_features, num_iterations):
    current_features = list(range(num_features))
    current_score = objective_function(current_features)

    for _ in range(num_iterations):
        neighbor = current_features.copy()
        if len(neighbor) > 1:
            feature_to_remove = random.choice(neighbor)
            neighbor.remove(feature_to_remove)
        else:
            features_not_in_subset = list(set(range(num_features)) - set(neighbor))
            if features_not_in_subset:
                feature_to_add = random.choice(features_not_in_subset)
                neighbor.append(feature_to_add)
        neighbor_score = objective_function(neighbor)
        if neighbor_score > current_score:
            current_features = neighbor
            current_score = neighbor_score

    return current_features, current_score

num_features = X_train.shape[1]
num_iterations = 100

selected_features, best_score = hill_climbing_feature_selection(num_features, num_iterations)

end_time = time.time()
elapsed_time = end_time - start_time

clf = SVC(kernel='linear', random_state=42) 
clf.fit(X_train[:, selected_features], y_train)
y_pred = clf.predict(X_test[:, selected_features])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

print("Selected Features:", selected_features)
print("Best Score:", best_score)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", confusion)
print("Time taken:", elapsed_time, "seconds")

KeyboardInterrupt: 

# -------------------------------------
# Naive Bayes


### Before GA and HC

In [18]:
X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = GaussianNB()

start_time = time.time()
clf.fit(X_train, y_train)
end_time = time.time()
execution_time = end_time - start_time

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Execution Time: {execution_time:.2f} seconds")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Execution Time: 0.01 seconds
Accuracy: 0.8927696078431373
Precision: 0.6044303797468354
Recall: 0.45913461538461536
F1-Score: 0.5218579234972678
Confusion Matrix:
 [[2723  125]
 [ 225  191]]
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94      2848
           1       0.60      0.46      0.52       416

    accuracy                           0.89      3264
   macro avg       0.76      0.71      0.73      3264
weighted avg       0.88      0.89      0.89      3264



### After applying GA 

In [20]:
X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

population_size = 50
generations = 100
mutation_rate = 0.01

def evaluate_features(features):
    clf = GaussianNB()
    clf.fit(X_train.iloc[:, features], y_train)
    y_pred = clf.predict(X_test.iloc[:, features])
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

population = [np.random.choice([0, 1], size=X_train.shape[1]) for _ in range(population_size)]

for generation in range(generations):
    fitness_scores = [evaluate_features(individual) for individual in population]
    num_parents = int(population_size * 0.2)
    parents = [population[i] for i in np.argsort(fitness_scores)[-num_parents:]]
    new_population = []
    for _ in range(population_size - num_parents):
        parent1 = population[np.random.choice(num_parents)]
        parent2 = population[np.random.choice(num_parents)]
        crossover_point = np.random.randint(0, X_train.shape[1])
        child = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
        mutation = np.random.rand(X_train.shape[1]) < mutation_rate
        child = (child + mutation) % 2
        new_population.append(child)
    population = np.vstack((parents, new_population))

best_individual = population[np.argmax(fitness_scores)]
selected_features = np.where(best_individual == 1)[0]

start_time_with_ga = time.time()
clf_with_ga = GaussianNB()
clf_with_ga.fit(X_train.iloc[:, selected_features], y_train)
y_pred_with_ga = clf_with_ga.predict(X_test.iloc[:, selected_features])
end_time_with_ga = time.time()

time_taken_with_ga = end_time_with_ga - start_time_with_ga

confusion_matrix_with_ga = confusion_matrix(y_test, y_pred_with_ga)

print("Time taken for Model with Genetic Algorithm Feature Selection:", time_taken_with_ga, "seconds")

print("\nConfusion Matrix for Model with Genetic Algorithm Feature Selection:")
print(confusion_matrix_with_ga)

accuracy_with_ga = accuracy_score(y_test, y_pred_with_ga)
precision_with_ga = precision_score(y_test, y_pred_with_ga)
recall_with_ga = recall_score(y_test, y_pred_with_ga)
f1_score_with_ga = f1_score(y_test, y_pred_with_ga)

print("\nModel with Genetic Algorithm Feature Selection Metrics:")
print("Accuracy (with GA):", accuracy_with_ga)
print("Precision (with GA):", precision_with_ga)
print("Recall (with GA):", recall_with_ga)
print("F1 Score (with GA):", f1_score_with_ga)

Time taken for Model with Genetic Algorithm Feature Selection: 0.004553318023681641 seconds

Confusion Matrix for Model with Genetic Algorithm Feature Selection:
[[2837   11]
 [ 237  179]]

Model with Genetic Algorithm Feature Selection Metrics:
Accuracy (with GA): 0.9240196078431373
Precision (with GA): 0.9421052631578948
Recall (with GA): 0.43028846153846156
F1 Score (with GA): 0.5907590759075908


### After applying HC

In [15]:
X = df.drop('Label', axis=1)
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

start_time = time.time()

def objective_function(features):
    clf = GaussianNB()
    clf.fit(X_train[:, features], y_train)
    y_pred = clf.predict(X_test[:, features])

    return accuracy_score(y_test, y_pred)

def hill_climbing_feature_selection(num_features, num_iterations):
    current_features = list(range(num_features))
    current_score = objective_function(current_features)

    for _ in range(num_iterations):
        neighbor = current_features.copy()
        if len(neighbor) > 1:
            feature_to_remove = random.choice(neighbor)
            neighbor.remove(feature_to_remove)
        else:
            features_not_in_subset = list(set(range(num_features)) - set(neighbor))
            if features_not_in_subset:
                feature_to_add = random.choice(features_not_in_subset)
                neighbor.append(feature_to_add)
        neighbor_score = objective_function(neighbor)
        if neighbor_score > current_score:
            current_features = neighbor
            current_score = neighbor_score

    return current_features, current_score

num_features = X_train.shape[1]
num_iterations = 100

selected_features, best_score = hill_climbing_feature_selection(num_features, num_iterations)

end_time = time.time()
elapsed_time = end_time - start_time

clf = GaussianNB()
clf.fit(X_train[:, selected_features], y_train)
y_pred = clf.predict(X_test[:, selected_features])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

print("Selected Features:", selected_features)
print("Best Score:", best_score)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", confusion)
print("Time taken:", elapsed_time, "seconds")

Selected Features: [0, 1, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 27, 28, 29]
Best Score: 0.9264705882352942
Accuracy: 0.9264705882352942
Precision: 0.9277485727928918
Recall: 0.9264705882352942
F1 Score: 0.9147725303016352
Confusion Matrix:
 [[2837   11]
 [ 229  187]]
Time taken: 0.307614803314209 seconds


### After applying GA

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

population_size = 50
generations = 100
mutation_rate = 0.01

def evaluate_features(features):
    clf = GaussianNB()
    clf.fit(X_train.iloc[:, features], y_train)
    y_pred = clf.predict(X_test.iloc[:, features])
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

population = [np.random.choice([0, 1], size=X_train.shape[1]) for _ in range(population_size)]

for generation in range(generations):
    fitness_scores = [evaluate_features(individual) for individual in population]
    num_parents = int(population_size * 0.2)
    parents = [population[i] for i in np.argsort(fitness_scores)[-num_parents:]]
    new_population = []
    for _ in range(population_size - num_parents):
        parent1 = population[np.random.choice(num_parents)]
        parent2 = population[np.random.choice(num_parents)]
        crossover_point = np.random.randint(0, X_train.shape[1])
        child = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
        mutation = np.random.rand(X_train.shape[1]) < mutation_rate
        child = (child + mutation) % 2
        new_population.append(child)
    population = np.vstack((parents, new_population))

best_individual = population[np.argmax(fitness_scores)]
selected_features = np.where(best_individual == 1)[0]

start_time_with_ga = time.time()
clf_with_ga = GaussianNB()
clf_with_ga.fit(X_train.iloc[:, selected_features], y_train)
y_pred_with_ga = clf_with_ga.predict(X_test.iloc[:, selected_features])
end_time_with_ga = time.time()

time_taken_with_ga = end_time_with_ga - start_time_with_ga

confusion_matrix_with_ga = confusion_matrix(y_test, y_pred_with_ga)

print("Time taken for Model with Genetic Algorithm Feature Selection:", time_taken_with_ga, "seconds")

print("\nConfusion Matrix for Model with Genetic Algorithm Feature Selection:")
print(confusion_matrix_with_ga)

accuracy_with_ga = accuracy_score(y_test, y_pred_with_ga)
precision_with_ga = precision_score(y_test, y_pred_with_ga)
recall_with_ga = recall_score(y_test, y_pred_with_ga)
f1_score_with_ga = f1_score(y_test, y_pred_with_ga)

print("\nModel with Genetic Algorithm Feature Selection Metrics:")
print("Accuracy (with GA):", accuracy_with_ga)
print("Precision (with GA):", precision_with_ga)
print("Recall (with GA):", recall_with_ga)
print("F1 Score (with GA):", f1_score_with_ga)

Time taken for Model with Genetic Algorithm Feature Selection: 0.0034110546112060547 seconds

Confusion Matrix for Model with Genetic Algorithm Feature Selection:
[[2835   13]
 [ 401   15]]

Model with Genetic Algorithm Feature Selection Metrics:
Accuracy (with GA): 0.8731617647058824
Precision (with GA): 0.5357142857142857
Recall (with GA): 0.036057692307692304
F1 Score (with GA): 0.06756756756756756


# Decision Trees

### Before GA and HC

In [13]:
X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier()

start_time = time.time()
clf.fit(X_train, y_train)
end_time = time.time()
execution_time = end_time - start_time

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Execution Time: {execution_time:.2f} seconds")
print("Accuracy:", accuracy)
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Execution Time: 0.18 seconds
Accuracy: 0.9451593137254902
Precision: 0.7925925925925926
Recall: 0.7716346153846154
F1-Score: 0.781973203410475
Confusion Matrix:
 [[2764   84]
 [  95  321]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.97      0.97      2848
           1       0.79      0.77      0.78       416

    accuracy                           0.95      3264
   macro avg       0.88      0.87      0.88      3264
weighted avg       0.94      0.95      0.94      3264



### After applying Genetic Algorithm

In [15]:
X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

population_size = 50
generations = 100
mutation_rate = 0.01

def evaluate_features(features):
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train.iloc[:, features], y_train)
    y_pred = clf.predict(X_test.iloc[:, features])
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

population = [np.random.choice([0, 1], size=X_train.shape[1]) for _ in range(population_size)]

for generation in range(generations):
    fitness_scores = [evaluate_features(individual) for individual in population]
    num_parents = int(population_size * 0.2)
    parents = np.argsort(fitness_scores)[-num_parents]
    parents = parents.tolist()
    new_population = []
    for _ in range(population_size - num_parents):
        parent1 = population[np.random.choice(parents)]
        parent2 = population[np.random.choice(parents)]
        crossover_point = np.random.randint(0, X_train.shape[1])
        child = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
        mutation = np.random.rand(X_train.shape[1]) < mutation_rate
        child = (child + mutation) % 2
        new_population.append(child)
    population = np.vstack((population[parents], new_population))

best_individual = population[np.argmax(fitness_scores)]
selected_features = np.where(best_individual == 1)[0]

start_time_with_ga = time.time()
clf_with_ga = DecisionTreeClassifier(random_state=42)
clf_with_ga.fit(X_train.iloc[:, selected_features], y_train)
y_pred_with_ga = clf_with_ga.predict(X_test.iloc[:, selected_features])
end_time_with_ga = time.time()

time_taken_with_ga = end_time_with_ga - start_time_with_ga

confusion_matrix_with_ga = confusion_matrix(y_test, y_pred_with_ga)

print("Time taken for Model with Genetic Algorithm Feature Selection:", time_taken_with_ga, "seconds")

print("\nConfusion Matrix for Model with Genetic Algorithm Feature Selection:")
print(confusion_matrix_with_ga)

accuracy_with_ga = accuracy_score(y_test, y_pred_with_ga)
precision_with_ga = precision_score(y_test, y_pred_with_ga)
recall_with_ga = recall_score(y_test, y_pred_with_ga)
f1_score_with_ga = f1_score(y_test, y_pred_with_ga)

print("\nModel with Genetic Algorithm Feature Selection Metrics:")
print("Accuracy (with GA):", accuracy_with_ga)
print("Precision (with GA):", precision_with_ga)
print("Recall (with GA):", recall_with_ga)
print("F1 Score (with GA):", f1_score_with_ga)

Time taken for Model with Genetic Algorithm Feature Selection: 0.10251116752624512 seconds

Confusion Matrix for Model with Genetic Algorithm Feature Selection:
[[2751   97]
 [ 102  314]]

Model with Genetic Algorithm Feature Selection Metrics:
Accuracy (with GA): 0.9390318627450981
Precision (with GA): 0.7639902676399026
Recall (with GA): 0.7548076923076923
F1 Score (with GA): 0.7593712212817413


### After applying HC 

In [22]:
X = df.drop('Label', axis=1)
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

def objective_function(features):
    start_time = time.time()
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train[:, features], y_train)
    y_pred = clf.predict(X_test[:, features])
    end_time = time.time()

    return accuracy_score(y_test, y_pred), start_time, end_time

def hill_climbing_feature_selection(num_features, num_iterations):
    current_features = list(range(num_features))
    current_score, start_time, _ = objective_function(current_features)  # Get start_time
    
    for _ in range(num_iterations):
        neighbor = current_features.copy()
        if len(neighbor) > 1:
            feature_to_remove = random.choice(neighbor)
            neighbor.remove(feature_to_remove)
        else:
            features_not_in_subset = list(set(range(num_features)) - set(neighbor))
            if features_not_in_subset:
                feature_to_add = random.choice(features_not_in_subset)
                neighbor.append(feature_to_add)
        neighbor_score, _, end_time = objective_function(neighbor)  # Get end_time
        if neighbor_score > current_score:
            current_features = neighbor
            current_score = neighbor_score
            elapsed_time = end_time - start_time  # Calculate elapsed_time

    return current_features, current_score, elapsed_time  # Return elapsed_time

num_features = X_train.shape[1]
num_iterations = 100

selected_features, best_score, elapsed_time = hill_climbing_feature_selection(num_features, num_iterations)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train[:, selected_features], y_train)
y_pred = clf.predict(X_test[:, selected_features])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

print("Selected Features:", selected_features)
print("Best Score:", best_score)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", confusion)
print("Time taken:", elapsed_time, "seconds")



Selected Features: [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29]
Best Score: 0.9509803921568627
Accuracy: 0.9322916666666666
Precision: 0.9342922223446857
Recall: 0.9322916666666666
F1 Score: 0.922399009493971
Confusion Matrix:
 [[2840    8]
 [ 213  203]]
Time taken: 0.6461198329925537 seconds


### ----------------------------------------------------------------------

# Logistic Regression

### Before applying GA and HC

In [17]:
X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression()

start_time = time.time()
clf.fit(X_train, y_train)
end_time = time.time()
execution_time = end_time - start_time

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Execution Time: {execution_time:.2f} seconds")
print("Accuracy:", accuracy)
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Execution Time: 3.44 seconds
Accuracy: 0.9283088235294118
Precision: 0.9789473684210527
Recall: 0.44711538461538464
F1-Score: 0.6138613861386139
Confusion Matrix:
 [[2844    4]
 [ 230  186]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96      2848
           1       0.98      0.45      0.61       416

    accuracy                           0.93      3264
   macro avg       0.95      0.72      0.79      3264
weighted avg       0.93      0.93      0.92      3264



### After applying GA 

In [30]:
X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

population_size = 50
generations = 100
mutation_rate = 0.01

def evaluate_features(features):
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train.iloc[:, features], y_train)
    y_pred = clf.predict(X_test.iloc[:, features])
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

population = [np.random.choice([0, 1], size=X_train.shape[1]) for _ in range(population_size)]

for generation in range(generations):
    fitness_scores = [evaluate_features(individual) for individual in population]
    num_parents = int(population_size * 0.2)
    parents = np.argsort(fitness_scores)[-num_parents]
    parents = parents.tolist()
    new_population = []
    for _ in range(population_size - num_parents):
        parent1 = population[np.random.choice(parents)]
        parent2 = population[np.random.choice(parents)]
        crossover_point = np.random.randint(0, X_train.shape[1])
        child = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
        mutation = np.random.rand(X_train.shape[1]) < mutation_rate
        child = (child + mutation) % 2
        new_population.append(child)
    population = np.vstack((population[parents], new_population))

best_individual = population[np.argmax(fitness_scores)]
selected_features = np.where(best_individual == 1)[0]
start_time_with_ga = time.time()
clf_with_ga = LogisticRegression(random_state=42)
clf_with_ga.fit(X_train.iloc[:, selected_features], y_train)
y_pred_with_ga = clf_with_ga.predict(X_test.iloc[:, selected_features])
end_time_with_ga = time.time()

time_taken_with_ga = end_time_with_ga - start_time_with_ga

confusion_matrix_with_ga = confusion_matrix(y_test, y_pred_with_ga)

print("Time taken for Model with Genetic Algorithm Feature Selection:", time_taken_with_ga, "seconds")

print("\nConfusion Matrix for Model with Genetic Algorithm Feature Selection:")
print(confusion_matrix_with_ga)

accuracy_with_ga = accuracy_score(y_test, y_pred_with_ga)
precision_with_ga = precision_score(y_test, y_pred_with_ga)
recall_with_ga = recall_score(y_test, y_pred_with_ga)
f1_score_with_ga = f1_score(y_test, y_pred_with_ga)

print("\nModel with Genetic Algorithm Feature Selection Metrics (Logistic Regression):")
print("Accuracy (with GA):", accuracy_with_ga)
print("Precision (with GA):", precision_with_ga)
print("Recall (with GA):", recall_with_ga)
print("F1 Score (with GA):", f1_score_with_ga)

Time taken for Model with Genetic Algorithm Feature Selection: 0.09711575508117676 seconds

Confusion Matrix for Model with Genetic Algorithm Feature Selection:
[[2834   14]
 [ 228  188]]

Model with Genetic Algorithm Feature Selection Metrics (Logistic Regression):
Accuracy (with GA): 0.9258578431372549
Precision (with GA): 0.9306930693069307
Recall (with GA): 0.4519230769230769
F1 Score (with GA): 0.6084142394822006


### After applying HC

In [44]:
X = df.drop('Label', axis=1)
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

def objective_function(features):
    clf = LogisticRegression(random_state=42, max_iter=100000)  # Increased max_iter
    clf.fit(X_train[:, features], y_train)
    y_pred = clf.predict(X_test[:, features])
    return accuracy_score(y_test, y_pred)

def hill_climbing_feature_selection(num_features, num_iterations):
    current_features = list(range(num_features))
    current_score = objective_function(current_features)

    for _ in range(num_iterations):
        neighbor = current_features.copy()
        if len(neighbor) > 1:  
            feature_to_remove = random.choice(neighbor)
            neighbor.remove(feature_to_remove)
        else:
            features_not_in_subset = list(set(range(num_features)) - set(neighbor))
            if features_not_in_subset:
                feature_to_add = random.choice(features_not_in_subset)
                neighbor.append(feature_to_add)
        neighbor_score = objective_function(neighbor)
        if neighbor_score > current_score:
            current_features = neighbor
            current_score = neighbor_score

    return current_features, current_score

num_features = X_train.shape[1]
num_iterations = 100

# Feature Selection
start_time = time.time()
selected_features, best_score = hill_climbing_feature_selection(num_features, num_iterations)
end_time = time.time()
elapsed_time = end_time - start_time

# Classifier Training and Evaluation
clf = LogisticRegression(random_state=42)
clf.fit(X_train[:, selected_features], y_train)
y_pred = clf.predict(X_test[:, selected_features])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

# Print Results
print("Selected Features:", selected_features)
print("Best Score:", best_score)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", confusion)
print("Time taken:", elapsed_time, "seconds")


Selected Features: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Best Score: 0.928921568627451
Accuracy: 0.9286151960784313
Precision: 0.9331319078876861
Recall: 0.9286151960784313
F1 Score: 0.9164398739322334
Confusion Matrix:
 [[2846    2]
 [ 231  185]]
Time taken: 15.794579982757568 seconds


/Users/saikarthik/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# K nearest Neighbours 

### Before applying GA and HC

In [50]:

X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = KNeighborsClassifier(n_neighbors=5)  

start_time = time.time()
clf.fit(X_train, y_train)
end_time = time.time()
execution_time = end_time - start_time

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Execution Time: {execution_time:.2f} seconds")
print("Accuracy:", accuracy)
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Execution Time: 0.00 seconds
Accuracy: 0.9200367647058824
Precision: 0.8506787330316742
Recall: 0.4519230769230769
F1-Score: 0.5902668759811617
Confusion Matrix:
 [[2815   33]
 [ 228  188]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.99      0.96      2848
           1       0.85      0.45      0.59       416

    accuracy                           0.92      3264
   macro avg       0.89      0.72      0.77      3264
weighted avg       0.92      0.92      0.91      3264



### After applying GA

In [51]:

X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

population_size = 50
generations = 100
mutation_rate = 0.01

def evaluate_features(features):
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train.iloc[:, features], y_train)
    y_pred = clf.predict(X_test.iloc[:, features])
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

population = [np.random.choice([0, 1], size=X_train.shape[1]) for _ in range(population_size)]



for generation in range(generations):
    fitness_scores = [evaluate_features(individual) for individual in population]
    num_parents = int(population_size * 0.2)
    parents = np.argsort(fitness_scores)[-num_parents]
    parents = parents.tolist()  
    new_population = []
    for _ in range(population_size - num_parents):
        parent1 = population[np.random.choice(parents)]
        parent2 = population[np.random.choice(parents)]
        crossover_point = np.random.randint(0, X_train.shape[1])
        child = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
        mutation = np.random.rand(X_train.shape[1]) < mutation_rate
        child = (child + mutation) % 2
        new_population.append(child)
    population = np.vstack((population[parents], new_population))

best_individual = population[np.argmax(fitness_scores)]
selected_features = np.where(best_individual == 1)[0]

start_time_with_ga = time.time()
clf = KNeighborsClassifier(n_neighbors=5)  # Set the number of neighbors (e.g., 5)

start_time = time.time()
clf.fit(X_train.iloc[:, selected_features], y_train)
end_time = time.time()
execution_time = end_time - start_time

y_pred = clf.predict(X_test.iloc[:, selected_features])

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Execution Time:", execution_time, "seconds")
print("Accuracy:", accuracy)
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", conf_matrix)

Execution Time: 0.0015380382537841797 seconds
Accuracy: 0.9206495098039216
Precision: 0.852017937219731
Recall: 0.4567307692307692
F1 Score: 0.594679186228482
Confusion Matrix:
 [[2815   33]
 [ 226  190]]


### After applying HC

In [54]:
X = df.drop('Label', axis=1)
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

def objective_function(features):
    start_time = time.time()
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train[:, features], y_train)
    y_pred = clf.predict(X_test[:, features])
    end_time = time.time()
    elapsed_time = end_time - start_time  # Calculate elapsed time
    return accuracy_score(y_test, y_pred), elapsed_time

def hill_climbing_feature_selection(num_features, num_iterations):
    current_features = list(range(num_features))
    current_score = objective_function(current_features)

    for _ in range(num_iterations):
        neighbor = current_features.copy()
        if len(neighbor) > 1:  
            feature_to_remove = random.choice(neighbor)
            neighbor.remove(feature_to_remove)
        else:
            features_not_in_subset = list(set(range(num_features)) - set(neighbor))
            if features_not_in_subset:
                feature_to_add = random.choice(features_not_in_subset)
                neighbor.append(feature_to_add)
        neighbor_score = objective_function(neighbor)
        if neighbor_score > current_score:
            current_features = neighbor
            current_score = neighbor_score

    return current_features, current_score


num_features = X_train.shape[1]
num_iterations = 100

selected_features, best_score = hill_climbing_feature_selection(num_features, num_iterations)

knn = KNeighborsClassifier(n_neighbors=5)  
knn.fit(X_train[:, selected_features], y_train)
y_pred = knn.predict(X_test[:, selected_features])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

print("Selected Features:", selected_features)
print("Best Score:", best_score)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", confusion)
print("Time taken: "+str(elapsed_time))

Selected Features: [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29]
Best Score: (0.9546568627450981, 0.1556558609008789)
Accuracy: 0.9200367647058824
Precision: 0.9155921981017185
Recall: 0.9200367647058824
F1 Score: 0.9091209391582129
Confusion Matrix:
 [[2815   33]
 [ 228  188]]
Time taken: 15.794579982757568
